In [1]:
from dotenv import load_dotenv
import streamlit as st
import os
import google_api
from PIL import Image

load_dotenv(dotenv_path='./.env')
api_path = os.environ.get('API_KEY_PATH', '')
if os.path.exists(api_path):
    google_api.init_credentials(api_path)

In [2]:
# Import Required Libraries
import os, io
import json
import cv2
from google_api import ai_image_response as ai_image_response
import ipywidgets as widgets
from IPython.display import display, clear_output
from threading import Thread
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI

def get_gemini_repsonse(input_prompt,image):

    model = ChatVertexAI(
        model_name='gemini-1.5-pro',
        convert_system_message_to_human=True,
    )

    response=ai_image_response(model, image=image, text=input_prompt)

    return response
    
# Create Widgets
input_prompt_widget = widgets.Textarea(
    value="""You are an expert nutritionist where you need to see the food items from the image
    and calculate the total calories, also provide the details of every food item with calories intake
    in the below format:

    1. Item 1 - no of calories
    2. Item 2 - no of calories
    ----
    ----
    Finally, you can mention whether the food is healthy or not and also mention the percentage split
    of the ratio of carbohydrates, fats, fibers, sugar, and other things required in our diet.""",
    placeholder='Type your input prompt here',
    description='Prompt:',
    layout={'width': '600px', 'height': '200px'}
)

analysis_button = widgets.Button(description='Analyze')
stop_camera_button = widgets.Button(description='Stop Camera')
clear_button = widgets.Button(description='Clear')

output_label = widgets.Textarea(
    value='',
    placeholder='The analysis result will be displayed here...',
    description='Result:',
    layout={'width': '600px', 'height': '200px'},
    disabled=True  # Make the output label read-only
)

camera_view = widgets.Image()

# Initialize camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

def update_camera_view():
    while True:
        ret, frame = cap.read()
        if ret:
            # Convert the frame to RGB format
            #frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to bytes
            _, buffer = cv2.imencode('.jpg', frame)
            camera_view.value = buffer.tobytes()

# Define Analysis Function
def analyze_image(b):
    input_prompt = input_prompt_widget.value
    ret, frame = cap.read()
    if not ret:
        output_label.value = "Failed to capture image"
        return
    
    # Convert the captured frame to RGB
    #frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    

    print("sending.input_prompt..")
    output_label.value = 'The analysis result will be displayed here...'
    response = get_gemini_repsonse(input_prompt, pil_image)
    #   response = get_gemini_repsonse(multi_model, image_data["data"], input_prompt)
    #response=multi_model.generate_content([input_prompt,image_data[0]])
    print(response)
    #print(response)
    
    # Display response in output label
    output_label.value = f"Food Details:\n{response}"

def clear_analyze(b):
    output_label.value = ""

# Attach Button Handlers
analysis_button.on_click(analyze_image)
clear_button.on_click(clear_analyze)

# Display Layout
display(widgets.HBox([camera_view, input_prompt_widget]))
display(widgets.HBox([analysis_button, stop_camera_button, clear_button]))
display(output_label)

# Start the camera feed in a separate thread
camera_thread = Thread(target=update_camera_view)
camera_thread.daemon = True
camera_thread.start()


Textarea(value='', description='Result:', disabled=True, layout=Layout(height='200px', width='600px'), placeho…